In [18]:
import pandas as pd
from pathlib import Path
import os

# base_dir = Path("datasets")
base_dir = Path("/work/shibberu/share/MA384_Data_Mining_Projects_Winter_2025-26/SMART_failure_prediction/S.M.A.R.T/datasets")

# 22 - 25
# folders = [base_dir / "data_Q4_2022", base_dir / "data_Q4_2023",base_dir / "data_Q4_2024"] # Q4
# folders = [base_dir / "data_Q3_2024", base_dir / "data_Q3_2025",base_dir / "data_Q3_2023", base_dir / "data_Q3_2022"] # Q3
# folders = [base_dir / "data_Q2_2022", base_dir / "data_Q2_2023",base_dir / "data_Q2_2024", base_dir / "data_Q2_2025"] # Q2
# folders = [base_dir / "data_Q1_2022", base_dir / "data_Q1_2023",base_dir / "data_Q1_2024", base_dir / "data_Q1_2025"] # Q1

# 16-21
# folders = [base_dir / "data_Q4_2016", base_dir / "data_Q4_2017",base_dir / "data_Q4_2018", base_dir / "data_Q4_2019", base_dir / "data_Q4_2020", base_dir / "data_Q4_2021"] # Q4
# folders = [base_dir / "data_Q3_2016", base_dir / "data_Q3_2017",base_dir / "data_Q3_2018", base_dir / "data_Q3_2019", base_dir / "data_Q3_2020", base_dir / "data_Q3_2021"] # Q3
# folders = [base_dir / "data_Q2_2016", base_dir / "data_Q2_2017",base_dir / "data_Q2_2018", base_dir / "data_Q2_2019", base_dir / "data_Q2_2020", base_dir / "data_Q2_2021"] # Q2
# folders = [base_dir / "data_Q1_2016", base_dir / "data_Q1_2017",base_dir / "data_Q1_2018", base_dir / "data_Q1_2019", base_dir / "data_Q1_2020", base_dir / "data_Q1_2021"] # Q1

# 13-15
folders = [base_dir / "data_2013"/"2013", base_dir / "data_2014"/"2014",base_dir / "data_2015"/"2015"] # full years

csv_files = []
for f in folders:
    if not f.exists():
        raise FileNotFoundError(f"Missing folder: {f.resolve()}")
    csv_files.extend(sorted(f.glob("*.csv")))

print("Found CSV files:", len(csv_files))
print("Example:", csv_files[0] if csv_files else "NONE")

Found CSV files: 996
Example: /work/shibberu/share/MA384_Data_Mining_Projects_Winter_2025-26/SMART_failure_prediction/S.M.A.R.T/datasets/data_2013/2013/2013-04-10.csv


In [19]:
for f in folders:
    files = sorted(f.glob("*.csv"))
    print(f"{f.name}: {len(files)} files")


2013: 266 files
2014: 365 files
2015: 365 files


In [20]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from pathlib import Path
import pandas as pd
import numpy as np
import os, time

# cores and threads setting
try:
    cpu_workers = len(os.sched_getaffinity(0))
except Exception:
    cpu_workers = os.cpu_count() or 1

max_workers = min(cpu_workers, len(csv_files)) if csv_files else cpu_workers
max_workers = max_workers // 2
max_workers = max(1, max_workers)
print("Using processes:", max_workers)


TARGET_N = 100000 # this is a threshold for preventing load too many rows for too much RAM usage
MODEL_COL = "model" # we want to balance rows per model to make the feature dynamic
FAIL_COL  = "failure"

# upper limit of candidates per file to prevent too many data comes from single file
CAND_PER_FILE = 300

# upper limit of candidates per model per file
PER_MODEL_PER_FILE = 3   

# filter: minimum non-null fraction per row
MIN_NON_NULL_FRAC = 0.25


def _read_one_csv(abs_path: str, base_dir_str: str):
    """
     read one csv file and return candidate rows DataFrame
    1) filter failure==0
    2) filter rows with too many nulls
    3) per model, take top PER_MODEL_PER_FILE rows by non-null count
    4) finally, if still too many rows, take top CAND_PER_FILE by non-null count
    5) add _key column for tracking source file
    6) return the candidate DataFrame   
    """
    p = Path(abs_path)
    base = Path(base_dir_str)
    key = str(p.relative_to(base))

    df = pd.read_csv(p, low_memory=False)

    # basic check
    if FAIL_COL not in df.columns or MODEL_COL not in df.columns:
        return pd.DataFrame(columns=["_key", MODEL_COL, "_nonnull_cnt", "_nonnull_frac"])

    # failure == 0
    df = df[df[FAIL_COL] == 0]
    if df.empty:
        return pd.DataFrame(columns=["_key", MODEL_COL, "_nonnull_cnt", "_nonnull_frac"])

    df[MODEL_COL] = df[MODEL_COL].astype(str)
    df = df[df[MODEL_COL].notna()]
    if df.empty:
        return pd.DataFrame(columns=["_key", MODEL_COL, "_nonnull_cnt", "_nonnull_frac"])

    # compute non-null counts and fractions
    df["_nonnull_cnt"] = df.notna().sum(axis=1)
    df["_nonnull_frac"] = df["_nonnull_cnt"] / df.shape[1]

    # filter by non-null fraction
    df = df[df["_nonnull_frac"] >= MIN_NON_NULL_FRAC]
    if df.empty:
        return pd.DataFrame(columns=["_key", MODEL_COL, "_nonnull_cnt", "_nonnull_frac"])

    # per model limit, sampling based on models
    df = df.sort_values("_nonnull_cnt", ascending=False)

    df = (
        df.groupby(MODEL_COL, group_keys=False)
          .head(PER_MODEL_PER_FILE)
    )

    # another global limit per file
    if len(df) > CAND_PER_FILE:
        df = df.sort_values("_nonnull_cnt", ascending=False).head(CAND_PER_FILE)

    df["_key"] = key

    return df


# multiprocessed reading and filtering
t0 = time.time()
candidates = []

with ProcessPoolExecutor(max_workers=max_workers) as ex:
    futures = [
        ex.submit(_read_one_csv, str(p.resolve()), str(base_dir.resolve()))
        for p in csv_files
    ]
    for fut in as_completed(futures):
        cdf = fut.result()
        if cdf is not None and not cdf.empty:
            candidates.append(cdf)

if not candidates:
    raise RuntimeError("No candidates found: either no failure==0 rows, or filters are too strict.")

cand_df = pd.concat(candidates, axis=0, ignore_index=True)
print("Candidate rows:", len(cand_df), "from files:", cand_df["_key"].nunique())
print("Candidate unique models:", cand_df[MODEL_COL].nunique())
print("Seconds (read candidates):", round(time.time() - t0, 3))


groups = {}
for m, g in cand_df.groupby(MODEL_COL, sort=False):
    groups[m] = g.sort_values("_nonnull_cnt", ascending=False)

models = list(groups.keys())
num_models = len(models)

base_quota = TARGET_N // num_models
remainder = TARGET_N % num_models

selected_parts = []
selected_count = 0
leftover_pool = []

for i, m in enumerate(models):
    g = groups[m]
    quota = base_quota + (1 if i < remainder else 0)
    take_n = min(quota, len(g))
    if take_n > 0:
        selected_parts.append(g.head(take_n))
        selected_count += take_n
    if take_n < len(g):
        leftover_pool.append(g.iloc[take_n:])

# # 
# if selected_count < TARGET_N:
#     need = TARGET_N - selected_count
#     if leftover_pool:
#         rest = pd.concat(leftover_pool, axis=0, ignore_index=True)
#         rest = rest.sort_values("_nonnull_cnt", ascending=False).head(need)
#         selected_parts.append(rest)
#         selected_count += len(rest)

final_df = pd.concat(selected_parts, axis=0, ignore_index=True)

# prevent exceeding TARGET_N
if len(final_df) > TARGET_N:
    final_df = final_df.sort_values("_nonnull_cnt", ascending=False).head(TARGET_N).reset_index(drop=True)

print("Final rows:", len(final_df))
print("Final unique models:", final_df[MODEL_COL].nunique())
print("Model distribution (top 20):")
print(final_df[MODEL_COL].value_counts())


Using processes: 128
Candidate rows: 124027 from files: 712
Candidate unique models: 84
Seconds (read candidates): 17.847
Final rows: 83917
Final unique models: 84
Model distribution (top 20):
model
ST320LT007                 1191
ST4000DM000                1191
Hitachi HDS5C3030ALA630    1191
WDC WD1600AAJS             1191
WDC WD1600BPVT             1191
                           ... 
WDC WD10EALS                252
Hitachi HDT721010SLA360     217
WDC WD30EZRS                148
WDC WD2500JB                 38
SAMSUNG HD103UJ              33
Name: count, Length: 84, dtype: int64


In [21]:
MODEL_COL = "model"
TARGET_TOTAL = 6000   # final wanted total rows written to a csv

# sample down based on per-model quota
K = final_df[MODEL_COL].nunique()
q = int(np.ceil(TARGET_TOTAL / K))   
print("Unique models:", K)
print("Target total:", TARGET_TOTAL)
print("Per-model quota q:", q)

# sort by quality: non-null fraction first, then non-null count
dfq = final_df.sort_values(["_nonnull_frac", "_nonnull_cnt"], ascending=[False, False]).copy()

# for each model: take up to q rows; if not enough, take all
sampled_df = (
    dfq.groupby(MODEL_COL, group_keys=False)
       .head(q)
       .reset_index(drop=True)
)

vc = sampled_df[MODEL_COL].value_counts()
print("Sampled total (no backfill):", len(sampled_df))
print("Min/Max per model:", int(vc.min()), int(vc.max()))
print("Models below quota:", int((vc < q).sum()))
print("\nBottom 10 models by count:\n", vc.sort_values().head(10))


Unique models: 84
Target total: 6000
Per-model quota q: 72
Sampled total (no backfill): 5975
Min/Max per model: 33 72
Models below quota: 2

Bottom 10 models by count:
 model
SAMSUNG HD103UJ    33
WDC WD2500JB       38
ST320LT007         72
ST250LT007         72
ST4000DX000        72
ST1500DL003        72
ST3000DM001        72
ST33000651AS       72
ST2000DL003        72
ST6000DX000        72
Name: count, dtype: int64


In [22]:
sampled_df.to_csv("merged_13-15_failure0.csv", index=False)

In [ ]:
# df_fail1 = pd.read_csv("merged_Q2_22-25_failure1.csv")

# model_counts = df_fail1["model"].value_counts()
# model_ratio = df_fail1["model"].value_counts(normalize=True)

# print("Model counts:")
# print(model_counts)

# print("\nModel ratios:")
# print(model_ratio)

# print("\nTotal number of unique models:", df_fail1["model"].nunique())

# print("Number of columns:", df_fail1.shape[1])

Model counts:
model
ST12000NM0008                             525
ST8000NM0055                              462
HGST HUH721212ALN604                      452
TOSHIBA MG07ACA14TA                       442
ST4000DM000                               381
HGST HUH721212ALE604                      270
ST8000DM002                               243
TOSHIBA MG08ACA16TA                       233
ST16000NM001G                             215
ST14000NM001G                             158
ST12000NM001G                             157
TOSHIBA MG08ACA16TE                       117
ST14000NM0138                              89
ST12000NM0007                              83
WDC WUH722222ALE6L4                        79
WDC WUH721816ALE6L4                        70
ST10000NM0086                              67
TOSHIBA MG08ACA16TEY                       57
WDC WUH721414ALE6L4                        53
HGST HMS5C4040BLE640                       48
WDC WUH721816ALE6L0                        33
TOSHIBA MQ01AB

/tmp/ipykernel_2198885/3564807754.py:1: DtypeWarning: Columns (181) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fail1 = pd.read_csv("merged_Q2_22-25_failure1.csv")


In [ ]:
from pathlib import Path
import pandas as pd

base_dir = Path(".")
pattern = "merged_*_failure*.csv"

csv_files = sorted(base_dir.glob(pattern))
print("Found files:", len(csv_files))
for f in csv_files:
    print("  ", f.name)

dfs = []
for f in csv_files:
    df = pd.read_csv(f, low_memory=False)
    df["_source_file"] = f.name  
    dfs.append(df)

merged_df = pd.concat(dfs, axis=0, join="outer", ignore_index=True)

# delete auxiliary columns used during processing
cols_to_drop = ["_nonnull_cnt", "_nonnull_frac", "_key", "_source_file"]
merged_df = merged_df.drop(columns=cols_to_drop, errors="ignore")

print("Merged shape:", merged_df.shape)
merged_df.to_csv("merged_all.csv", index=False)

print("Saved (auxiliary columns removed)")



Found files: 18
   merged_13-15_failure0.csv
   merged_13-15_failure1.csv
   merged_Q1_16-21_failure1.csv
   merged_Q1_16_21_failure0.csv
   merged_Q1_22-25_failure1.csv
   merged_Q1_22_25_failure0.csv
   merged_Q2_16-21_failure1.csv
   merged_Q2_16_21_failure0.csv
   merged_Q2_22-25_failure1.csv
   merged_Q2_22_25_failure0.csv
   merged_Q3_16-21_failure1.csv
   merged_Q3_16_21_failure0.csv
   merged_Q3_22-25_failure1.csv
   merged_Q3_22_25_failure0.csv
   merged_Q4_16-21_failure1.csv
   merged_Q4_16_21_failure0.csv
   merged_Q4_22-25_failure1.csv
   merged_Q4_22_25_failure0.csv
Merged shape: (76542, 197)
Saved (auxiliary columns removed)


# Check merged files

In [5]:
df_merged = pd.read_csv("merged_all.csv", low_memory=False)
# print all columns to a txt file
with open("merged_all_columns.txt", "w") as f:
    for col in df_merged.columns:
        f.write(col + "\n") 
        

In [ ]:
from collections import Counter
import re


with open("merged_all_columns.txt", "r") as f:
    cols = [line.strip() for line in f if line.strip()]

# get SMART IDs
smart_nums = []
for c in cols:
    m = re.match(r"smart_(\d+)_", c)
    if m:
        smart_nums.append(int(m.group(1)))

cnt = Counter(smart_nums)

# filter appearances >= 3 (more than raw + normalized)
dup = {k: v for k, v in cnt.items() if v >= 3}

print("Total unique SMART IDs:", len(cnt))

if dup:
    print("SMART IDs appearing more than twice:")
    for k, v in sorted(dup.items()):
        print(f"smart_{k}: {v} times")
else:
    print("No SMART ID appears more than twice (raw + normalized only).")


Total unique SMART IDs: 93
No SMART ID appears more than twice (raw + normalized only).


In [7]:
# get number of rows for failure==0 and failure==1
df_merged = pd.read_csv("merged_all.csv", low_memory=False)
num_fail0 = (df_merged["failure"] == 0).sum()
num_fail1 = (df_merged["failure"] == 1).sum()
print("Number of rows with failure==0:", num_fail0)
print("Number of rows with failure==1:", num_fail1)

Number of rows with failure==0: 46416
Number of rows with failure==1: 30126
